In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import os,sys,time
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from datetime import datetime, timedelta 
from sklearn.metrics import mean_absolute_error
from collections import OrderedDict

In [ ]:
data = pd.read_csv("4200_C005_2019_03_03.tsv",sep = ",", names = ("kunag","matnr","date","quantity","price"),header = None)
data = data[data['quantity']>=0]
df = data.copy()
# data.head()

## Preprocess and Train Test split

In [ ]:
##n_series to find the individual series for a kunag,matnr combination
def n_series(input_df,kunag,matnr):
    #removing the negative value rows
    input_df = input_df[input_df["quantity"] >=0]    
    df = input_df.copy()
    
    #finding a single customer(kunag),material(matnr) combination on the basis of input provided
    n_df1 = df[(df["kunag"] == kunag) & (df["matnr"] == matnr)]
    
    #parsing the date column in a datetime format
    n_df1.date = n_df1.date.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))
    #sort date start to end
    n_df1 = n_df1.sort_values('date')
    n_df1.set_index('date',inplace=True)
    
    #sampling the data on weekly basis (index is set to date first in the above step to do weekly sampling
    weekly_resampled_data = n_df1.quantity.resample('W').sum() 
    weekly_resampled_data = weekly_resampled_data.replace(np.nan, 0)
    individual_series = weekly_resampled_data.to_frame() 

    individual_series["external3"] = individual_series["quantity"].rolling(3).sum().iloc[::]
    individual_series['external3'] = individual_series['external3'].shift(1)
    individual_series["external3"].fillna(0,inplace = True)
    
    individual_series["external6"] = individual_series["quantity"].rolling(6).sum().iloc[::]
    individual_series['external6'] = individual_series['external6'].shift(1)
    individual_series["external6"].fillna(0,inplace = True)
    
    #resetting index so that date can be used as column
    individual_series = individual_series.reset_index()
    return individual_series


#splitting the individual series basesd on the number of weeks (i, is for dyanamic point forecasts)
def train_test_split(input_df,kunag,matnr,i):
    data= n_series(input_df,kunag,matnr)
    train = data.iloc[2:-i]    
    test = data.iloc[-i:]
    return train,test

## Sarima WITHOUT external regressor

In [ ]:
def sarima(input_df, kunag,matnr,n,p,d,q):
    i = 0
    lst = []
    test1 = train_test_split(df,kunag,matnr,n)[1]
    y_hat_avg = test1.copy()
    start_date = str(test1["date"][:1])
    end_date = str(test1["date"][-1:])
    order = (p, d,q)
    for i in range(n,0,-1):
        train,test = train_test_split(df,kunag,matnr,i)
        dd= np.asarray(train["quantity"])
        
        ## WITHOUT external regressor
        fit1 = sm.tsa.statespace.SARIMAX(train["quantity"],order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n").fit()    
        
        pred = fit1.predict(1)
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst

    rms = sqrt(mean_squared_error(test1.quantity, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.quantity, y_hat_avg.pred_column)
    return y_hat_avg,rms,mae

## Sarima WITH external regressor

In [ ]:
def sarima_er(input_df, kunag,matnr,n,p,d,q):
    i = 0
    lst = []
    test1 = train_test_split(df,kunag,matnr,n)[1]
    y_hat_avg = test1.copy()
    start_date = str(test1["date"][:1])
    end_date = str(test1["date"][-1:])
    order = (p, d,q)
    for i in range(n,0,-1):
        train,test = train_test_split(df,kunag,matnr,i)
        dd= np.asarray(train["quantity"])

        ## with external regressor
        model1 = sm.tsa.statespace.SARIMAX(train["quantity"], exog = sm.add_constant(train["external"]),order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n")    
        fit1 = model1.fit()
        exog_pred = [[1, train["quantity"].iloc[-7:].sum()]]
        pred = fit1.forecast(1, exog=exog_pred)
#         pred = fit1.predict(1)
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst
    rms = sqrt(mean_squared_error(test1.quantity, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.quantity, y_hat_avg.pred_column)
    return y_hat_avg,rms,mae

In [ ]:
def sarima_er1(input_df, kunag,matnr,n,p,d,q):
    i = 0
    lst1 = []
    lst2 = []
    test1 = train_test_split(df,kunag,matnr,n)[1]
    y_hat_avg = test1.copy()
    start_date = str(test1["date"][:1])
    end_date = str(test1["date"][-1:])
    order = (p, d,q)
    for i in range(n,0,-1):
        train,test = train_test_split(df,kunag,matnr,i)
        dd= np.asarray(train["quantity"])

        ## with external regressor
        model1 = sm.tsa.statespace.SARIMAX(train["quantity"], exog = sm.add_constant(train["external3"]),order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n")    
        model2 = sm.tsa.statespace.SARIMAX(train["quantity"], exog = sm.add_constant(train["external6"]),order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n")    

        fit1 = model1.fit()
        fit2 = model2.fit()

        exog_pred1 = [[1, train["quantity"].iloc[-3:].sum()]]
        exog_pred2 = [[1, train["quantity"].iloc[-6:].sum()]]

        
        pred1 = fit1.forecast(1, exog=exog_pred1)
        pred2 = fit2.forecast(1, exog=exog_pred2)

        lst1.append(pred1.iloc[-1])
        lst2.append(pred2.iloc[-1])
        
        
    pd.DataFrame(lst1)
    pd.DataFrame(lst2)

    y_hat_avg['pred_column1']=lst1      
    y_hat_avg['pred_column2']=lst2

#     rms = sqrt(mean_squared_error(test1.quantity, y_hat_avg.pred_column))
#     mae = mean_absolute_error(test1.quantity, y_hat_avg.pred_column)
    return y_hat_avg

In [ ]:
# df = data.copy()
# sarima_er1(df, 500056565,100278,16,0,1,1)

## Plot Comparisions

In [ ]:
def plot(df,kunag,matnr,n):
        p,d,q = 0,1,1
        train,test1 = train_test_split(df,kunag,matnr,n)

        output1,rms1,mae1 = sarima(df,kunag,matnr,n,p,d,q)
#         output2,rms2,mae2 = sarima_er(df,kunag,matnr,n,p,d,q)
        
       ##combined freq 3 and 6 
        output = sarima_er1(df,kunag,matnr,n,p,d,q)

        
        plt.figure(figsize=(18,10))
        plt.plot(train.set_index("date")['quantity'], label='Train',marker = '.')
        plt.plot(test1.set_index("date")['quantity'], label='Test',marker = '.')
        plt.plot(output1.set_index("date")['pred_column'], label='arima' ,marker = '.')
#         plt.plot(output2.set_index("date")['pred_column'], label='arima_x' ,marker = '.')  
        
        ##plot for combined freq 3 and 6 
        plt.plot(output.set_index("date")['pred_column1'], label='arima_3' ,marker = '.')
        plt.plot(output.set_index("date")['pred_column2'], label='arima_6' ,marker = '.',color = "blue")  
        
        plt.legend(loc='best')
        plt.title("Arima_3_6_Comparision")
        plt.savefig(folder +"/" + 'Graph_{}.png'.format(index), format="PNG")  
#         plt.show()
        return

In [ ]:
# folder = 'model_graphs/'+"SARIMAX_3_7"
# if not os.path.exists(folder):
#         os.makedirs(folder)
# df = data.copy()
# bucket = pd.read_csv("bucket.csv")

# main_df = pd.DataFrame(columns = {"kunag","matnr","mae_arima","mae_sarimax"})
# cnt = 1
# n = 16
# for i in range (1,2):
#             start = time.time()
#             kunag = int(bucket["kunag"].iloc[i])
#             matnr = int(bucket["matnr"].iloc[i])
#             index = str(kunag) +"_"+ str(matnr)
#             print("count",cnt)
#             print("index : ",index)
            
#             #PLOT_COMPARISION
#             mae1,mae2 = plot(df,kunag,matnr,n)

#             result_df = pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
#                        "mae_arima" : [round(mae1,3)],
#                        "mae_sarimax" : [round(mae2,3)]}))

#             print(result_df)
#             main_df = main_df.append(result_df, ignore_index = True) 
#             export_csv = main_df.to_csv (r'model_graphs/SARIMAX_F16/results/arima_x_comp().csv',sep = ",", index = None, header=True)
#             end = time.time()
#             cnt+=1
#             print("Time Taken : ",end - start)  
            

#### For combined 3 and 6

In [ ]:
folder = 'model_graphs/'+"SARIMAX_3_6"
if not os.path.exists(folder):
        os.makedirs(folder)
df = data.copy()
bucket = pd.read_csv("bucket.csv")

# main_df = pd.DataFrame(columns = {"kunag","matnr","mae_arima","mae_sarimax"})
cnt = 1
n = 16
for i in range (201,501):
            start = time.time()
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            print("count",cnt)
            print("index : ",index)
            
            #PLOT_COMPARISION
            output =  plot(df,kunag,matnr,n)

#             result_df = pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
#                        "mae_arima" : [round(mae1,3)],
#                        "mae_sarimax" : [round(mae2,3)]}))

#             print(result_df)
#             main_df = main_df.append(result_df, ignore_index = True) 
#             export_csv = main_df.to_csv (r'model_graphs/SARIMAX_F16/results/arima_x_comp().csv',sep = ",", index = None, header=True)
            end = time.time()
            cnt+=1
            print("Time Taken : ",end - start)  
            

## Csv Analysis

In [ ]:
!ls model_graphs/SARIMAX_F3/results

In [ ]:
df1 = pd.read_csv("model_graphs/SARIMAX_F3/results/arima_x_comp(0,100).csv",sep = ",")
df2 = pd.read_csv("model_graphs/SARIMAX_F3/results/arima_x_comp(100,200).csv", sep = ",")
df3 = pd.read_csv("model_graphs/SARIMAX_F3/results/arima_x_comp(200,500).csv", sep = ",")
df4 = pd.read_csv("model_graphs/SARIMAX_F3/results/arima_x_comp(500,1000).csv", sep = ",")

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

In [ ]:
df = pd.concat([df1,df3, df2,df4],ignore_index=True)

In [ ]:
df.head()

In [ ]:
minimum = df.idxmin(axis=1)

In [ ]:
type(minimum)

In [ ]:
a = 0
b = 0

for i in range(0,len(minimum)):
    if(minimum[i]=="mae_arima"):
        a+=1
    elif(minimum[i]=="mae_sarimax"):
        b+=1

print(a)
print(b)


In [ ]:
df.sum(axis = 0, skipna = True) 

In [ ]:
arima__ = 530
arima_x3__ = 470
mae_arima     = 8.218440e+02
mae_sarimax3   = 8.143150e+02

## Multiple regressors

In [ ]:
##n_series to find the individual series for a kunag,matnr combination
def n_series1(input_df,kunag,matnr):
    #removing the negative value rows
    input_df = input_df[input_df["quantity"] >=0]    
    df = input_df.copy()
    
    #finding a single customer(kunag),material(matnr) combination on the basis of input provided
    n_df1 = df[(df["kunag"] == kunag) & (df["matnr"] == matnr)]
    
    #parsing the date column in a datetime format
    n_df1.date = n_df1.date.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))
    #sort date start to end
    n_df1 = n_df1.sort_values('date')
    n_df1.set_index('date',inplace=True)
    
    #sampling the data on weekly basis (index is set to date first in the above step to do weekly sampling
    weekly_resampled_data = n_df1.quantity.resample('W').sum() 
    weekly_resampled_data = weekly_resampled_data.replace(np.nan, 0)
    individual_series = weekly_resampled_data.to_frame() 

    individual_series["external1"] = individual_series["quantity"].rolling(1).sum().iloc[::]
    individual_series['external1'] = individual_series['external1'].shift(1)
    individual_series["external1"].fillna(0,inplace = True)
    
    individual_series["external2"] = individual_series["quantity"].rolling(2).sum().iloc[::]
    individual_series['external2'] = individual_series['external2'].shift(1)
    individual_series["external2"].fillna(0,inplace = True)
    
    individual_series["external3"] = individual_series["quantity"].rolling(3).sum().iloc[::]
    individual_series['external3'] = individual_series['external3'].shift(1)
    individual_series["external3"].fillna(0,inplace = True)
    
    #resetting index so that date can be used as column
    individual_series = individual_series.reset_index()
    return individual_series


#splitting the individual series basesd on the number of weeks (i, is for dyanamic point forecasts)
def train_test_split1(input_df,kunag,matnr,i):
    data= n_series1(input_df,kunag,matnr)
    train = data.iloc[3:-i]    
    test = data.iloc[-i:]
    return train,test

In [ ]:
# train_test_split1(df,500056565,100278,16)

In [ ]:
def sarimax_multix(input_df, kunag,matnr,n,p,d,q):
    i = 0
    lst = []
    test1 = train_test_split1(df,kunag,matnr,n)[1]
    y_hat_avg = test1.copy()
    start_date = str(test1["date"][:1])
    end_date = str(test1["date"][-1:])
    order = (p, d,q)
    for i in range(n,0,-1):
        train,test = train_test_split1(df,kunag,matnr,i)
        dd= np.asarray(train["quantity"])

        ## with external regressor
        model1 = sm.tsa.statespace.SARIMAX(train["quantity"], exog = sm.add_constant(train[['external1','external2','external3']]),order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n")    
        fit1 = model1.fit()
        x1 = train["quantity"].iloc[-1:].sum()
        x2 = train["quantity"].iloc[-2:].sum()
        x3 = train["quantity"].iloc[-3:].sum()
    

#         exog_pred = [[1, train["quantity"].iloc[-7:].sum()]]
        exog_pred = [[1, x1,x2,x3]]

        pred = fit1.forecast(1, exog=exog_pred)
#         pred = fit1.predict(1)
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst
    rms = sqrt(mean_squared_error(test1.quantity, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.quantity, y_hat_avg.pred_column)
    return y_hat_avg,rms,mae

In [ ]:
def plot(df,kunag,matnr,n):
        p,d,q = 0,1,1
        train,test1 = train_test_split(df,kunag,matnr,n)

        output1,rms1,mae1 = sarima(df,kunag,matnr,n,p,d,q)
#         output2,rms2,mae2 = sarima_er(df,kunag,matnr,n,p,d,q)
        
       ##combined freq 3 and 6 
        output,rms,mae = sarimax_multix(df,kunag,matnr,n,p,d,q)

        
        plt.figure(figsize=(18,10))
        plt.plot(train.set_index("date")['quantity'], label='Train',marker = '.')
        plt.plot(test1.set_index("date")['quantity'], label='Test',marker = '.')
        plt.plot(output1.set_index("date")['pred_column'], label='arima' ,marker = '.')
#         plt.plot(output2.set_index("date")['pred_column'], label='arima_x' ,marker = '.')  
        
        ##plot for combined freq 3 and 6 
        plt.plot(output.set_index("date")['pred_column'], label='arimax_(1,2,3)' ,marker = '.')
#         plt.plot(output.set_index("date")['pred_column2'], label='arima_6' ,marker = '.',color = "blue")  
        
        plt.legend(loc='best')
        plt.title("Arima_3_6_Comparision")
        plt.savefig(folder +"/" + 'Graph_{}.png'.format(index), format="PNG")  
#         plt.show()
        return mae1,mae

In [ ]:
folder = 'model_graphs/'+"SARIMAX_123"
if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
df = data.copy()
bucket = pd.read_csv("bucket.csv")

main_df = pd.DataFrame(columns = {"kunag","matnr","mae_arima","mae_sarimax123"})
cnt = 1
n = 16
for i in range (601,1001):
            start = time.time()
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            print("count",cnt)
            print("index : ",index)
            
            #PLOT_COMPARISION
            mae1,mae = plot(df,kunag,matnr,n)

            result_df = pd.DataFrame(OrderedDict({"kunag" : [kunag],"matnr" : [matnr],
                       "mae_arima" : [round(mae1,3)],
                       "mae_sarimax123" : [round(mae,3)]}))

            print(result_df)
            main_df = main_df.append(result_df, ignore_index = True) 
            export_csv = main_df.to_csv (r'model_graphs/SARIMAX_123/results/arima_x_comp(600-400).csv',sep = ",", index = None, header=True)
            end = time.time()
            cnt+=1
            print("Time Taken : ",end - start)  
            